In [248]:
import pandas as pd
import numpy as np

from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource, CDSView, GroupFilter
from bokeh.plotting import figure

output_notebook()

Loading BokehJS ...

In [302]:
# Load data

# KCO
alltime_kco_df = pd.read_csv('Woo_KCO_All_Time_5419_results 2020 May 1.csv')

# #KP
alltime_kp_df = pd.read_csv('Woo_KP_All_time_3594_results_2020_May_1.csv')

#Regex
oldregex_df = pd.read_csv('MIDs_by_UA_before_April_9.csv')
regex_df = pd.read_csv('MIDs_by_UA_since_April_8.csv')
newest_df = pd.read_csv('MIDs_by_UA_May_3.csv')
may4_df = pd.read_csv('MIDs_by_UA_May_4.csv')
may4_extra_df = pd.read_csv('MIDs_by_UA_May_4_17_missing.csv')
may4_evening_df = pd.read_csv('MIDs_by_UA_May_4_1216.csv')

regex_df = pd.concat([regex_df, newest_df, may4_df, may4_extra_df])

alltime_kco = set(alltime_kco_df['merchant_id'])
alltime_kp = set(alltime_kp_df['merchant_id'])
alltime_both = alltime_kco | alltime_kp

print(f'distinct count of MIDS all time:  {len(alltime_both)}')
mids_trans_since_apr8 = set(regex_df['merchant_id'])
mids_before_april9 = set(oldregex_df['merchant_id'])
today = set(may4_evening_df['merchant_id'])

regex_df = pd.concat([regex_df, may4_evening_df])

print(f'MIDS that have NOT trans since Apr 8, deduped :  {len(alltime_both - mids_trans_since_apr8)}')

before_today = mids_trans_since_apr8 | alltime_both | mids_before_april9
new_today =   today - before_today
print(f'MIDS that are new today :  {len(new_today)}')
#was 2119

#Merchant Contact List
sf_mer_master_df = pd.read_csv('Merchant contact master file - All merchants.csv')

#oldregex_df.info()
#print(regex_df[regex_df['merchant_id'] == 'K521293'].iloc[0,1])
#regex_df[regex_df['merchant_id'] == 'K521293']
#may4_extra_df[may4_extra_df['merchant_id'] == 'K521293']
#may4_extra_df['merchant_id'] == 'K521293'
#may4_extra_df


distinct count of MIDS all time:  8983
MIDS that have NOT trans since Apr 8, deduped :  2062
MIDS that are new today :  32


In [250]:
#sf_mer_master_df

In [251]:
# Prep dataframe
regex_df['timestamp'] = regex_df['timestamp'].str[-24:-2]
regex_df['merchant_id'] = regex_df['merchant_id'].str[0:7]
regex_df['timestamp'] = pd.to_datetime(regex_df['timestamp'])
regex_df.columns = ['timestamp', 'UA', 'merchant_id', 'purchase_country']
regex_df.set_index('timestamp', inplace=True)
regex_df.sort_index(ascending=False, inplace=True)

# Prep old dataframe
oldregex_df['timestamp'] = oldregex_df['timestamp'].str[-24:-2]
oldregex_df['merchant_id'] = oldregex_df['merchant_id'].str[0:7]
oldregex_df['timestamp'] = pd.to_datetime(oldregex_df['timestamp'])
oldregex_df.columns = ['timestamp', 'UA', 'merchant_id', 'purchase_country']
oldregex_df.set_index('timestamp', inplace=True)
oldregex_df.sort_index(ascending=False, inplace=True)

oldregex_df.drop_duplicates('merchant_id', inplace=True)

# filtering out from old regex if exists in newer regex
oldregex_df = oldregex_df[~oldregex_df['merchant_id'].isin(regex_df['merchant_id'])]
#oldregex_df


In [252]:
regex_df.drop_duplicates('merchant_id', inplace=True)
#regex_df
#regex_df.info()
regex_df = pd.concat([regex_df, oldregex_df])
#regex_df.info()

In [253]:
# Extract Regex's into df columns

#reg_KCO1 = regex_df.UA.str.extract(r"WordPress.* https://(?P<merchant_url>.*) - Woo.* KCO:(?P<kco_version>.*) - PHP .*", expand=True)
#reg_KCO2 = reg_KCO1.kco_version.str.extract(r"(?P<major>.*)\.(?P<minor>.*)\.(?P<patch>.*)", expand=True)

reg_KCO1 = regex_df.UA.str.extract(r"WordPress.* (?P<merchant_url>(http|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])).* - KCO:(?P<kco_version>.*) - PHP .*", expand=True)
reg_KCO2 = reg_KCO1.kco_version.str.extract(r"(?P<major>.*)\.(?P<minor>.*)\.(?P<patch>.*)", expand=True)

reg_KP1 = regex_df.UA.str.extract(r"WordPress.* (?P<merchant_url>(http|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])).* - KP:(?P<kp_version>.*) - PHP .*", expand=True)
reg_KP2 = reg_KP1.kp_version.str.extract(r"(?P<major>.*)\.(?P<minor>.*)\.(?P<patch>.*)", expand=True)

regex_df = pd.concat([regex_df, reg_KCO1, reg_KCO2], axis=1)


In [254]:
#regex_df


In [255]:
# Combine KCO and KP columns
regex_df['merchant_url'] = regex_df['merchant_url'].combine_first(reg_KP1['merchant_url'])
regex_df['kp_version'] = reg_KP1['kp_version']
regex_df['major'] = regex_df['major'].combine_first(reg_KP2['major'])
regex_df['minor'] = regex_df['minor'].combine_first(reg_KP2['minor'])
regex_df['patch'] = regex_df['patch'].combine_first(reg_KP2['patch'])

#filter out entries that don't match regex
regex_df = regex_df[regex_df.major.notnull()]

regex_df = regex_df[['UA', 'merchant_id', 'purchase_country', 'merchant_url', 'kco_version', 'kp_version', 'major', 'minor', 'patch']]
regex_df['major'] = regex_df.major.astype('int32')
regex_df['minor'] = regex_df.minor.astype('int32')
regex_df['patch'] = regex_df.patch.astype('int32')

#regex_df


,UA,merchant_id,purchase_country,merchant_url,kco_version,kp_version,major,minor,patch
timestamp,,,,,,,,,
2020-05-05 00:13:15.060,WordPress/4.9.14; https://vigoshop.de - KP:1.6...,K503072,DE,https://vigoshop.de,NaN,1.6.4,1,6,4
2020-05-05 00:13:06.850,WordPress/5.2.2; https://www.bananabeauty.de -...,K510393,DE,https://www.bananabeauty.de,NaN,2.0.7,2,0,7
2020-05-05 00:13:02.860,WordPress/5.1.2; https://offer.heartechlabs.co...,N102846,US,https://offer.heartechlabs.com/01/klarna,NaN,2.0.7,2,0,7
2020-05-05 00:12:52.390,WordPress/5.4.1; https://landstil.com - WooCom...,K504845,NO,https://landstil.com,2.0.13,NaN,2,0,13
2020-05-05 00:12:48.340,WordPress/5.2.2; https://estrid.com/no/ - WooC...,K511654,NO,https://estrid.com/no/,1.11.6,NaN,1,11,6
...,...,...,...,...,...,...,...,...,...
2020-02-04 08:17:58.630,WordPress/5.3.2; https://shophuuthinh.com - KP...,N606689,US,https://shophuuthinh.com,NaN,2.0.4,2,0,4
2020-02-04 08:11:11.560,WordPress/5.1.4; https://unginvest.no - WooCom...,K611896,NO,https://unginvest.no,1.9.1,NaN,1,9,1
2020-02-04 02:27:31.110,WordPress/5.3.2; https://www.faszinierendschoe...,K653657,DE,https://www.faszinierendschoen.de,NaN,1.9.0,1,9,0


In [256]:
regex_df[regex_df.major.isnull()]

,UA,merchant_id,purchase_country,merchant_url,kco_version,kp_version,major,minor,patch
timestamp,,,,,,,,,


In [257]:
good_major0 = regex_df[regex_df['kco_version'] == '1.11.8']
good_major1 = regex_df[(regex_df['kco_version'].notnull()) & (regex_df['major'] == 1) & (regex_df['minor'] < 9)]
good_major2 = regex_df[(regex_df['kco_version'].notnull()) & (regex_df['major'] == 2) & (regex_df['minor'] >= 0) & (regex_df['patch'] >= 10)]

good_kp_major1 = regex_df[(regex_df['kp_version'].notnull()) & (regex_df['major'] == 1) & (regex_df['minor'] < 7)]
good_kp_major2 = regex_df[(regex_df['kp_version'].notnull()) & (regex_df['major'] == 2) & (regex_df['minor'] >= 0) & (regex_df['patch'] >= 7)]

is_good_df = pd.concat([good_major0, good_major1, good_major2, good_kp_major1, good_kp_major2])

bad_major1 = regex_df[(regex_df['kco_version'].notnull()) & (regex_df['major'] == 1) & (regex_df['major'] != 2) & (regex_df['minor'] >= 9) & ~((regex_df['minor'] == 11) & (regex_df['patch'] == 8))]
bad_major2 = regex_df[(regex_df['kco_version'].notnull()) & (regex_df['major'] == 2) & (regex_df['minor'] == 0) & (regex_df['patch'] < 10)]

bad_kp_major1 = regex_df[(regex_df['kp_version'].notnull()) & (regex_df['major'] == 1) & (regex_df['minor'] >= 7)]
bad_kp_major2 = regex_df[(regex_df['kp_version'].notnull()) & (regex_df['major'] == 2) & (regex_df['minor'] == 0) & (regex_df['patch'] < 7)]

is_bad_df = pd.concat([bad_major1, bad_major2, bad_kp_major1, bad_kp_major2])

In [258]:
#bad_major2

In [259]:
regex_df[regex_df.merchant_url.isnull()]

,UA,merchant_id,purchase_country,merchant_url,kco_version,kp_version,major,minor,patch
timestamp,,,,,,,,,


In [260]:
#master_df.shape

In [261]:
# Truncate the merchant_id to contain only one ID (since All KP has some doubled MIDs)
#left_kp_df['merchant_id'] = left_kp_df['merchant_id'].str[0:7]
# master_df['merchant_id'] = master_df['merchant_id'].str[0:7]

In [262]:
# Dedup the merchant_id in the master file
# id_before = master_df.merchant_id.count()
# print(f'Merchant ID field length BEFORE de_dup = {id_before}')
# master_df.merchant_id.drop_duplicates(inplace=True)
# id_after = master_df.merchant_id.count()
# print(f'Merchant ID field length AFTER de_dup = {id_after}')
# print(f'Removed {id_before - id_after} duplicate merchant ids from master file.')

In [263]:
# | rename "message.request.body.purchase_country" AS purchase_country
# | rename "message.request.headers.User-Agent" AS ua
# | rex field=ua "WordPress.* https://(?<merchant_name>.*) - Woo.* KCO:(?<kco_version>.*) - PHP .*"
# | rex field=ua "https://(?<merchant_name>.com -)"
# | rex field=kco_version "(?<major>.*)\.(?<minor>.*)\.(?<patch>.*)"

# | eval bad_version= if((major=1 AND NOT (minor=11 AND patch>=8)) OR (major=2 AND minor=0 AND patch<=9),1,0)
# | eval good_version= if((major=1 AND NOT (minor=11 AND patch>=8)) OR (major=2 AND minor=0 AND patch<=9),0,1)

# Kristi's edits

# | rex field=ua "WordPress.* http.://(?<merchant_name>.*) - Woo.* KCO:(?<kco_version>.*) - PHP .*"
# | rex field=kco_ua "https://(?<merchant_name>.com -)"
# |rex field=kco_version "(?<major>.*)\.(?<minor>.*)\.(?<patch>.*)"
# | eval bad_version= if((major=1 AND NOT (minor=11 AND patch>=8)) OR (major=2 AND minor=0 AND patch<=9),1,0)
# | eval good_version= if((major=1 AND NOT (minor=11 AND patch>=8)) OR (major=2 AND minor=0 AND patch<=9),0,1)

# |rex field=ua "WordPress.* http.://(?<merchant_name>.*) - Woo.*  KP:(?<kp_version>.*) - PHP .*"
# | rex field=ua "https://(?<merchant_name>.com -)"
# |rex field=kp_version "(?<major>.*)\.(?<minor>.*)\.(?<patch>.*)"
# | eval bad_version= if((major=1 ) OR (major=2 AND minor=0 AND patch<=6),1,0)
# | eval good_version= if((major=1 ) OR (major=2 AND minor=0 AND patch<=6),0,1)

In [264]:
# Create Sets
all_mids = set(regex_df['merchant_id'])
is_good = set(is_good_df['merchant_id'])
is_bad = set(is_bad_df['merchant_id'])
is_never_good = is_bad - is_good
krokedil_desist_df = pd.read_csv('MIDs updated from Krokedil.csv')
krokedil_desist = set(krokedil_desist_df['merchant_id'])

print(f'All MIDs = {len(all_mids)}')
print(f'Good MIDs = {len(is_good)}')
print(f'Bad MIDs = {len(is_bad)}')
print(f'Intersection Good and Bad = {len(is_good & is_bad)}')
print(f'All - Good = {len(all_mids - is_good)}')
print(f'All - Bad = {len(all_mids - is_bad)}')
print(f'Is Never Good = {len(is_never_good)}')
print(f'Is Never Good - Krokedil Desist = {len(is_never_good - krokedil_desist)}')



All MIDs = 8191
Good MIDs = 5758
Bad MIDs = 2433
Intersection Good and Bad = 0
All - Good = 2433
All - Bad = 5758
Is Never Good = 2433
Is Never Good - Krokedil Desist = 2433


In [265]:
# Merge in splunk data
spl_good_df = (regex_df[regex_df.merchant_id.isin(pd.Series(list(is_good)))]
               .sort_index(ascending=False)
               .drop_duplicates('merchant_id'))
spl_good_df['kristi_good'] = 'Good'

spl_bad_df = (regex_df[regex_df.merchant_id.isin(pd.Series(list(is_never_good)))]
              .sort_index(ascending=False)
              .drop_duplicates('merchant_id'))
spl_bad_df['kristi_good'] = 'Bad'

spl_df = pd.concat([spl_good_df, spl_bad_df])
#spl_df

,UA,merchant_id,purchase_country,merchant_url,kco_version,kp_version,major,minor,patch,kristi_good
timestamp,,,,,,,,,,
2020-05-05 00:13:15.060,WordPress/4.9.14; https://vigoshop.de - KP:1.6...,K503072,DE,https://vigoshop.de,NaN,1.6.4,1,6,4,Good
2020-05-05 00:13:06.850,WordPress/5.2.2; https://www.bananabeauty.de -...,K510393,DE,https://www.bananabeauty.de,NaN,2.0.7,2,0,7,Good
2020-05-05 00:13:02.860,WordPress/5.1.2; https://offer.heartechlabs.co...,N102846,US,https://offer.heartechlabs.com/01/klarna,NaN,2.0.7,2,0,7,Good
2020-05-05 00:12:52.390,WordPress/5.4.1; https://landstil.com - WooCom...,K504845,NO,https://landstil.com,2.0.13,NaN,2,0,13,Good
2020-05-05 00:12:41.880,WordPress/5.4; https://www.gmcrafts.co.uk - Wo...,K514003,GB,https://www.gmcrafts.co.uk,NaN,2.0.7,2,0,7,Good
...,...,...,...,...,...,...,...,...,...,...
2020-02-04 08:17:58.630,WordPress/5.3.2; https://shophuuthinh.com - KP...,N606689,US,https://shophuuthinh.com,NaN,2.0.4,2,0,4,Bad
2020-02-04 08:11:11.560,WordPress/5.1.4; https://unginvest.no - WooCom...,K611896,NO,https://unginvest.no,1.9.1,NaN,1,9,1,Bad
2020-02-04 02:27:31.110,WordPress/5.3.2; https://www.faszinierendschoe...,K653657,DE,https://www.faszinierendschoen.de,NaN,1.9.0,1,9,0,Bad


In [266]:
# Merge Salesforce
sf_splk_full_outer_df = pd.merge(sf_mer_master_df, spl_df, how='outer', left_on='merchant_id', right_on='merchant_id')

sf_splk_full_outer_df['Volume 2018-2020 (SEK)'].fillna('0', inplace=True)
sf_splk_full_outer_df['Volume 2018-2020 (SEK)'] = sf_splk_full_outer_df.loc[:,'Volume 2018-2020 (SEK)'].str.replace('.', ',')
sf_splk_full_outer_df['Volume 2018-2020 (SEK)'] = sf_splk_full_outer_df.loc[:,'Volume 2018-2020 (SEK)'].str.replace(',', '').astype('int32')


sf_splk_full_outer_df = sf_splk_full_outer_df.sort_values('Volume 2018-2020 (SEK)', ascending=False)

In [277]:
#sf_splk_full_outer_df.info()
#sf_splk_full_outer_df
sf_splk_full_outer_df.kristi_good.value_counts(dropna=False)
sf_splk_full_outer_df[sf_splk_full_outer_df['kristi_good'].isnull()]

,Root business entity ID FINAL,merchant_id,Email Final,Merchant Country Final,Phone number (SF),Store website,Plug-in version,Account owner,Account owner team,Volume 2018-2020 (SEK),...,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,UA,purchase_country,merchant_url,kco_version,kp_version,kristi_good
8309,NaN,K69867,NaN,NaN,NaN,NaN,Bad,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8817,NaN,K68443,NaN,NaN,NaN,NaN,Bad,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [268]:
sf_splk_full_outer_df.drop(['major', 'minor', 'patch'], inplace=True, axis=1)

sf_splk_full_outer_df.to_csv('SF_Splunk_Full_Outer.csv')

In [269]:
# print(f'Size All KCO Merchants all time = {len(left)}')
# print(f'Size Good KCO Since Apr 8 = {len(right)}')
     
# # Get Intersection    
# # left_inter_right = left & right
# # left_inter_right_size = len(left & right)
# # print(f'Intersection {left_inter_right_size}')
# # inter_df = pd.DataFrame(left_inter_right, columns=['merchant_id'])
# # inter_df.to_csv('intersection.csv', index=False)

# # left minus right KCO
# left_minus_right = left - right
# left_minus_right_size = len(left_minus_right)
# print(f'Bad KCO Since Apr 8 = {left_minus_right_size}')
# left_m_right_df = pd.DataFrame(left_minus_right, columns=['merchant_id'])
# left_m_right_df.to_csv('Bad KCO.csv', index=False)

# print(f'Size All KP Merchants all time = {len(left_kp)}')
# print(f'Size Good KP Since Apr 8 = {len(right_kp)}')
      
# # left minus right KP
# leftkp_minus_right = left_kp - right_kp
# leftkp_minus_right_size = len(leftkp_minus_right)
# print(f'Bad KP Since Apr 8 = {leftkp_minus_right_size}')
# left_kp_m_right_df = pd.DataFrame(leftkp_minus_right, columns=['merchant_id'])
# left_kp_m_right_df.to_csv('Bad KP.csv', index=False)     
      
# # add KCO + KP
# both = left_minus_right | leftkp_minus_right     
# both_size = len(both)
# print(f'Both KCO & KP Bad = {both_size}')
      
# # now minus Krokedil desist
# both_minus_Krokedil = both - KrokedilDesist
# both_minus_Krokedil_size = len(both_minus_Krokedil)
# print(f'Bad KCO & KP minus Krokedil desist = {both_minus_Krokedil_size}')
# both_minus_Krokedil_df = pd.DataFrame(both_minus_Krokedil, columns=['merchant_id'])
# both_minus_Krokedil_df.to_csv('Final Bad KCO & KP.csv', index=False)      

# # left minus right2
# #left_minus_right2 = left_minus_right - right2
# #left_minus_right2_size = len(left_minus_right2 - right2)
# #print(f'Remainder minus May 1 {left_minus_right2_size}')

# # left minus right3
# #left_minus_right3 = left_minus_right2 - right3
# #left_minus_right3_size = len(left_minus_right3 - right3)
# #print(f'Remainder minus May 2 {left_minus_right3_size}')

# # left minus right4
# # left_minus_right4 = left_minus_right3 - right4
# # left_minus_right4_size = len(left_minus_right4 - right4)
# # print(f'Remainder minus May 3 {left_minus_right4_size}')

# # BroaderGood intersection Remainder 
# # bad_inter_broader_good =  left_minus_right4 & broader_good
# # bad_inter_broader_good_size = len(left_minus_right4 & broader_good)
# # print(f'Remainder inter Broader Good, {bad_inter_broader_good_size}')

# # left minus broader_good
# # bad_minus_broader_good = left_minus_right4 - broader_good
# # bad_minus_broader_good_size = len(left_minus_right4 - broader_good)
# # print(f'Remainder minus Broader Good {bad_minus_broader_good_size}')

# # right minus left
# # right_minus_left = right - left
# # right_minus_left_size = len(right - left)
# # print(f'Good minus All {right_minus_left_size}')
# # right_m_left_df = pd.DataFrame(right_minus_left, columns=['merchant_id'])
# # right_m_left_df.to_csv('right_m_left.csv', index=False)


In [270]:
# Save lists to CSV
# inter_df = pd.DataFrame(left_inter_right, columns=['merchant_id'])
# inter_df.to_csv('intersection.csv', index=False)

# left_m_right_df = pd.DataFrame(left_minus_right, columns=['merchant_id'])
# left_m_right_df.to_csv('Not Confirmed Good.csv', index=False)

# #left_m_right2_df = pd.DataFrame(left_minus_right2, columns=['merchant_id'])
# #left_m_right2_df.to_csv('Remainder_m_May1.csv', index=False)

# #left_m_right3_df = pd.DataFrame(left_minus_right3, columns=['merchant_id'])
# #left_m_right3_df.to_csv('Remainder_m_May2.csv', index=False)

# #left_m_right4_df = pd.DataFrame(left_minus_right4, columns=['merchant_id'])
# #left_m_right4_df.to_csv('Remainder_m_May3.csv', index=False)

# #bad_inter_broader_good_df = pd.DataFrame(bad_inter_broader_good, columns=['merchant_id'])
# #bad_inter_broader_good_df.to_csv('bad_inter_broader_good.csv', index=False)

# #bad_minus_broader_good_df = pd.DataFrame(bad_minus_broader_good, columns=['merchant_id'])
# #bad_minus_broader_good_df.to_csv('Bad_m_broader_good.csv', index=False)

# left_minus_Krokedil_df = pd.DataFrame(left_minus_Krokedil, columns=['merchant_id'])
# left_minus_Krokedil_df.to_csv('Final Bad.csv', index=False)

# right_m_left_df = pd.DataFrame(right_minus_left, columns=['merchant_id'])
# right_m_left_df.to_csv('right_m_left.csv', index=False)
